# Data Access Examples

The data modules provide a way to access the underlying data and transform it to facilitate analysis. This includes:

- Data retrieval using SQL
- Creation of mutation matrices
- Creation of vectors of dependent variables

In [1]:
%matplotlib inline

import init
from microbepy.common import constants as cn
from microbepy.common import isolate
from microbepy.common.range_constraint import RangeConstraint
from microbepy.common.study_context import nextStudyContext
from microbepy.common.study_context import StudyContext
from microbepy.common import util
from microbepy.data.model_data_provider import ModelDataProvider, ModelDataDualProvider

import copy
import numpy as np
import pandas as pd

## Data Model

Microbepy assumes that data are organized in terms of:

- Isolate: This is the microbial community (a generalization of the usual definition of isolate)
- Mutation: Changes to the genome
- Culture: Phenotype information obtained from a culture

An isolate is described in terms of the following:

- Evolutionary line (often, just line)
- Transfer time (time at which the isolate was obtained)
- End point dilution (abbreviated EPD) or None if not applicable
- Clone (an index of same genome organisms on a plate) or None if not applicable
- Species or None if multiple species
- Experiment (just CI in these data)

For example, the isolate HA3.152.10.01.D.CI has evoluationary line HA3, transfer 152, EPD 10, clone 1, and species DVH.

A mutation is specified by its an affected gene (if applicable), position in the genome, nucleotides in the reference genome, and the changed nucleotides. For example, DVU2451.2555217.CA.C is a mutation in the DVU gene DVU2451 at position 2555217 that changes the nucelotides CA to C.

The culture is a string that uniquely identifies each single or paired incubation of microbes.

These data are combined into a single table called ``genotype_phenotype``. The keys are the isolate (key_isolate), mutation (key_mutation), and culture (key_culture). The table ``genotype`` only contains information related to isolates and mutation. Details of the columns in these tables can be found in microbepy.common.constants.py.

Mutations may be at different ``granularities`` as indicated by the names of columns that contain mutation values.

- KEY_MUTATION - identifies a single change to the genome
- GGENE_ID - mutations to the same gene are aggregated in that the gene is mutated if at least one mutation is present for a nucleotide in the genome; intergenic mutations are identified by position
- GENE_ID - only considers genes
- COG, EC - higher level classifications of genes

The database file is specified in the ``.microbepy`` directory (in the user's home directory) in the file ``config.py``.

## SQL Access to Data

The function ``util.readSQL`` queries the data repository and returns a dataframe with the columns specified in the query.

In [2]:
sql_cmd = "select key_isolate, key_mutation, key_culture from genotype_phenotype where transfer = 152"
df = util.readSQL(sql_cmd)
df.head()

,key_isolate,key_mutation,key_culture
0,HR2.152.01.*.*.*,DVU0001.412.CCCCCCTCGCAGCCCCC.CCCCCC,C127
1,HR2.152.01.*.*.*,DVU0001.412.CCCCCCTCGCAGCCCCC.CCCCCC,C128
2,HR2.152.01.*.*.*,DVU0001.412.CCCCCCTCGCAGCCCCC.CCCCCC,C129
3,HR2.152.01.*.*.*,DVU0001.412.CCCCCCTCGCAGCCCCC.CCCCCC,C130
4,HR2.152.01.*.*.*,DVU0001.412.CCCCCCTCGCAGCCCCC.CCCCCC,C278


Note that many of the results are for end point dilutions since the clone and species are "\*" (None). To obtain true isolates, the query can be modified.

In [3]:
sql_cmd = """
select key_isolate, key_mutation, key_culture from genotype_phenotype
     where transfer = 152 and species in ('D', 'M')
"""
df = util.readSQL(sql_cmd)
df.head()

,key_isolate,key_mutation,key_culture
0,HR2.152.05.01.D.CI,DVU0001.412.CCCCCCTCGCAGCCCCC.CCCCCC,C647
1,HR2.152.05.01.D.CI,DVU0001.412.CCCCCCTCGCAGCCCCC.CCCCCC,C648
2,HR2.152.05.01.D.CI,DVU0001.412.CCCCCCTCGCAGCCCCC.CCCCCC,C649
3,HR2.152.05.01.D.CI,DVU0001.412.CCCCCCTCGCAGCCCCC.CCCCCC,C168
4,HR2.152.05.01.D.CI,DVU0001.412.CCCCCCTCGCAGCCCCC.CCCCCC,C169


To obtain phenotype information, we had columns for ``rate`` and ``yield``.

In [4]:
sql_cmd = """
select key_isolate, key_mutation, key_culture, rate, yield from genotype_phenotype
     where transfer = 152 and species in ('D', 'M')
"""
df = util.readSQL(sql_cmd)
df.head()

,key_isolate,key_mutation,key_culture,rate,yield
0,HR2.152.05.01.D.CI,DVU0001.412.CCCCCCTCGCAGCCCCC.CCCCCC,C647,0.025899,0.426375
1,HR2.152.05.01.D.CI,DVU0001.412.CCCCCCTCGCAGCCCCC.CCCCCC,C648,0.028403,0.438955
2,HR2.152.05.01.D.CI,DVU0001.412.CCCCCCTCGCAGCCCCC.CCCCCC,C649,0.026304,0.441246
3,HR2.152.05.01.D.CI,DVU0001.412.CCCCCCTCGCAGCCCCC.CCCCCC,C168,0.030093,0.519520
4,HR2.152.05.01.D.CI,DVU0001.412.CCCCCCTCGCAGCCCCC.CCCCCC,C169,0.029688,0.525941


## Mutation Matrix

The mutation matrix is a dataframe representation of a matrix.
The columns are values of mutations at a specified granularity (e.g., KEY_MUTATION, GGENE_ID).
Values are either 1 (mutation is present) or 0 (mutation is absent).
The row index is the community in which the mutation was found. For paired-isolate cultures,
a tuple is used.

Dependent variables growth and yield (the phenotypes) are provided as a dataframe as well, with the same
row index structure as the associated mutation matrix.

The class ``ModelDataProvider`` is used to construct the mutation matrix and the dependent variables(s).

In computational studies, we often iterate through a range of values for the dependent variable, choice of mutation column, and evolutionary line. Rather than writing similar loops repeatedly, we have created a python class called StudyContext that specifies the values to use.

In [5]:
# Mutation matrix for
provider = ModelDataProvider(StudyContext(depvar=cn.RATE, mutation_column=cn.KEY_MUTATION))
provider.do()  # Initialize the matrices
print(provider.df_X.head())
provider.df_y.head()

                                       DVH__.1007324.A.G  DVH__.1010502.T.C  \
key_isolate_dvh    key_isolate_mmp                                            
HA2.152.01.01.D.CI HA2.152.01.01.M.CI                0.0                0.0   
HA2.152.01.02.D.CI HA2.152.01.02.M.CI                0.0                0.0   
HA2.152.01.03.D.CI HA2.152.01.03.M.CI                0.0                0.0   
HA2.152.05.01.D.CI HA2.152.05.01.M.CI                0.0                0.0   
HA2.152.05.03.D.CI HA2.152.05.03.M.CI                0.0                0.0   

                                       DVH__.1259554.C.T  DVH__.1297045.C.T  \
key_isolate_dvh    key_isolate_mmp                                            
HA2.152.01.01.D.CI HA2.152.01.01.M.CI                0.0                0.0   
HA2.152.01.02.D.CI HA2.152.01.02.M.CI                0.0                0.0   
HA2.152.01.03.D.CI HA2.152.01.03.M.CI                0.0                0.0   
HA2.152.05.01.D.CI HA2.152.05.01.M.CI              

,,rate
key_isolate_dvh,key_isolate_mmp,
HA2.152.01.01.D.CI,HA2.152.01.01.M.CI,1.344413
HA2.152.01.02.D.CI,HA2.152.01.02.M.CI,-0.434761
HA2.152.01.03.D.CI,HA2.152.01.03.M.CI,-1.059424
HA2.152.05.01.D.CI,HA2.152.05.01.M.CI,-0.657592
HA2.152.05.03.D.CI,HA2.152.05.03.M.CI,-0.427354


In [6]:
provider = ModelDataProvider(StudyContext(depvar=cn.YIELD, mutation_column=cn.GGENE_ID))
provider.do()  # Initialize the matrices
print(provider.df_X.head())
provider.df_y.head()

                                       DVH__.1007324  DVH__.1010502  \
key_isolate_dvh    key_isolate_mmp                                    
HA2.152.01.01.D.CI HA2.152.01.01.M.CI            0.0            0.0   
HA2.152.01.02.D.CI HA2.152.01.02.M.CI            0.0            0.0   
HA2.152.01.03.D.CI HA2.152.01.03.M.CI            0.0            0.0   
HA2.152.05.01.D.CI HA2.152.05.01.M.CI            0.0            0.0   
HA2.152.05.03.D.CI HA2.152.05.03.M.CI            0.0            0.0   

                                       DVH__.1259554  DVH__.1297045  \
key_isolate_dvh    key_isolate_mmp                                    
HA2.152.01.01.D.CI HA2.152.01.01.M.CI            0.0            0.0   
HA2.152.01.02.D.CI HA2.152.01.02.M.CI            0.0            0.0   
HA2.152.01.03.D.CI HA2.152.01.03.M.CI            0.0            0.0   
HA2.152.05.01.D.CI HA2.152.05.01.M.CI            0.0            0.0   
HA2.152.05.03.D.CI HA2.152.05.03.M.CI            0.0            0.0   

    

,,yield
key_isolate_dvh,key_isolate_mmp,
HA2.152.01.01.D.CI,HA2.152.01.01.M.CI,0.666848
HA2.152.01.02.D.CI,HA2.152.01.02.M.CI,0.626393
HA2.152.01.03.D.CI,HA2.152.01.03.M.CI,0.066244
HA2.152.05.01.D.CI,HA2.152.05.01.M.CI,-0.177557
HA2.152.05.03.D.CI,HA2.152.05.03.M.CI,-0.286660


Note that values of dependent variables are standardized. So a value of 0 is the mean of the observations.

In [7]:
# Extracting the communities from mutation matrix
print('\n'.join(["%s, %s" % (d, m) for d, m in provider.df_X.index.tolist()]))

HA2.152.01.01.D.CI, HA2.152.01.01.M.CI
HA2.152.01.02.D.CI, HA2.152.01.02.M.CI
HA2.152.01.03.D.CI, HA2.152.01.03.M.CI
HA2.152.05.01.D.CI, HA2.152.05.01.M.CI
HA2.152.05.03.D.CI, HA2.152.05.03.M.CI
HA2.152.08.01.D.CI, HA2.152.08.01.M.CI
HA2.152.08.03.D.CI, HA2.152.08.03.M.CI
HA2.152.09.01.D.CI, HA2.152.09.01.M.CI
HA2.152.09.02.D.CI, HA2.152.09.02.M.CI
HA2.152.09.03.D.CI, HA2.152.09.03.M.CI
HR2.152.01.01.D.CI, HR2.152.01.01.M.CI
HR2.152.01.02.D.CI, HR2.152.01.02.M.CI
HR2.152.01.03.D.CI, HR2.152.01.03.M.CI
HR2.152.05.01.D.CI, HR2.152.05.01.M.CI
HR2.152.05.02.D.CI, HR2.152.05.02.M.CI
HR2.152.05.03.D.CI, HR2.152.05.03.M.CI
HR2.152.10.01.D.CI, HR2.152.10.01.M.CI
HR2.152.10.02.D.CI, HR2.152.10.02.M.CI
HR2.152.10.03.D.CI, HR2.152.10.03.M.CI
UE3.152.02.01.D.CI, UE3.152.02.01.M.CI
UE3.152.02.02.D.CI, UE3.152.02.02.M.CI
UE3.152.02.03.D.CI, UE3.152.02.03.M.CI
UE3.152.03.01.D.CI, UE3.152.03.01.M.CI
UE3.152.03.02.D.CI, UE3.152.03.02.M.CI
UE3.152.03.03.D.CI, UE3.152.03.03.M.CI
UE3.152.09.01.D.CI, UE3.1